In [1]:
from data import get_dataloader
import torch.optim as optim
import yaml
from DB.db_detector import SegDetector
from DB.db_mobilenet import mobilenet_v3_large
from DB.losses import L1BalanceCELoss
from DB.representers import SegDetectorRepresenter
import torch
import numpy as np
from torch import nn

class temp_model(nn.Module):
    def __init__(self):
        super(temp_model,self).__init__()
        self.back = mobilenet_v3_large().to(device)
        self.neck = SegDetector(adaptive=True,in_channels=[24, 40, 112, 960],k= 50).to(device)
    def forward(self,x):
        return self.neck(self.back(x))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_tensor_type(torch.FloatTensor)

with open('./conf/train.yaml') as f:
    conf = yaml.load(f,Loader=yaml.FullLoader)
loader = get_dataloader(conf['Dataset'])

model = temp_model()
model = torch.nn.DataParallel(model).to(device)

filtered_parameters = []
params_num = []
for p in filter(lambda p: p.requires_grad, model.parameters()):
    filtered_parameters.append(p)
    params_num.append(np.prod(p.size()))
print('Trainable params num : ', sum(params_num))

loss = L1BalanceCELoss()
optimizer = optim.Adam(filtered_parameters, lr=0.0005, betas=(0.9, 0.999))
representer = SegDetectorRepresenter()

Trainable params num :  6685890


In [2]:
for data in loader:
    optimizer.zero_grad()
    result = model(data['image'].to(torch.float))
    losses,metric = loss(result,data)
    losses.backward()
    optimizer.step()
    print(losses)

/root/storage/BKH/DET/data/make_border_map.py:95: RuntimeWarning: invalid value encountered in true_divide
  (2 * np.sqrt(square_distance_1 * square_distance_2))
/root/storage/BKH/DET/data/make_border_map.py:102: RuntimeWarning: invalid value encountered in less
  square_distance_1, square_distance_2))[cosin < 0]


tensor(11.6581, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.9654, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f95a2d00c80> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f95a2a88488> (for post_execute):


KeyboardInterrupt: 

In [ ]:
data['shape']= [(1024,768)*len(data['image'])]
representer.represent(data,result)

In [ ]:
res

In [ ]:
import matplotlib.pyplot as plt
print(result.keys())
print(data.keys())

ind = 2
plt.figure(figsize=(20,20))
plt.imshow(result['binary'].cpu().detach().numpy()[0,0])
plt.show()
plt.imshow(result['thresh'].cpu().detach().numpy()[0,0])
plt.show()
plt.imshow(result['thresh_binary'].cpu().detach().numpy()[0,0])
plt.show()
plt.figure(figsize=(20,20))
plt.imshow(data['image'][0].permute(1,2,0).contiguous().cpu().detach().numpy())
plt.show()
plt.figure(figsize=(20,20))

plt.imshow(data['thresh_map'].cpu().detach().numpy()[0])
plt.show()
plt.figure(figsize=(20,20))
plt.imshow(data['thresh_mask'].cpu().detach().numpy()[0])
plt.show()

plt.imshow(data['gt'].cpu().detach().numpy()[0,0])
plt.show()
plt.imshow(data['mask'].cpu().detach().numpy()[0])
plt.show()

plt.figure(figsize=(20,20))
plt.imshow(np.tile(data['thresh_mask'].cpu().detach().numpy()[0][:,:,np.newaxis],3) * data['image'][0].permute(1,2,0).contiguous().cpu().detach().numpy())
plt.show()

In [ ]:
from shapely.geometry import Polygon

idx1 = 3
idx2 = 2
def temp(data,idx1,idx2):
    rdata = data.copy() 
    td = rdata[idx1].copy()
    rdata[idx1] = rdata[idx2]
    rdata[idx2] = td
    print(data,'\n &',rdata)
    return rdata

Polygon(temp(data['polygons'][0][0],idx1,idx2))

In [ ]:
temp(data['polygons'][0][0],idx1,idx2)

In [ ]:
from data import DetectionData

dataset = DetectionData(conf['Dataset'])
test = dataset[3]
for i in test.keys():
    print(test[i].shape,i)

In [ ]:
def calc_point(points):
    max_x = max(points[:,0])
    min_x = min(points[:,0]) 
    max_y = max(points[:,1]) 
    min_y = min(points[:,1]) 
    diff_x = max_x-min_x
    diff_y = max_y-min_y
    for point in points:
        if point[0]==min_x and point[1]==min_y:
            return diff_x,diff_y,point
        
import matplotlib.patches as patches


fig, ax = plt.subplots(figsize=(30,30))
ax.imshow(data['image'][0].permute(1,2,0).cpu().detach().numpy())

for word in data['polygons'][0]:
    diff_x,diff_y,point = calc_point(word)
    rect = patches.Rectangle(point,diff_x,diff_y,facecolor='none', linewidth=1, edgecolor='r')
    ax.add_patch(rect)
plt.show()

In [ ]:
data['gt'].shape

In [ ]:
data['image'].shape

In [ ]:
import pickle

with open('/root/storage/BKH/DET/train_data/dataset_v1.3/02/orig_boxes/60.pkl','rb') as f:
    tt = pickle.load(f)

In [ ]:
tt

In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import pickle
import os

In [ ]:
'''
path = '/root/storage/BKH/DET/train_data/dataset_v1.3/02/'
box_folder = 'orig_boxes'
img_folder = 'orig_texts'

for filename in os.listdir(os.path.join(path,box_folder)):
    file = filename.split('.')[0]
    image = Image.open(os.path.join(path,img_folder,file+'.png'))
    img = np.asarray(image)/255
                       
    with open(os.path.join(path,box_folder,file+'.pkl'),'rb') as f:
        boxes = pickle.load(f)
    new_list = list()
    for i,word in enumerate(boxes):
        new_list.append(word['box'][np.newaxis,:,:])
        '''

In [ ]:

def calc_point(points):
    max_x = max(points[:,0])
    min_x = min(points[:,0]) 
    max_y = max(points[:,1]) 
    min_y = min(points[:,1]) 
    diff_x = max_x-min_x
    diff_y = max_y-min_y
    for point in points:
        if point[0]==min_x and point[1]==min_y:
            return diff_x,diff_y,point
        
import matplotlib.patches as patches


fig, ax = plt.subplots(figsize=(30,30))
ax.imshow(test['image'])

for word in test['polygons']:
    diff_x,diff_y,point = calc_point(word)
    rect = patches.Rectangle(point,diff_x,diff_y,facecolor='none', linewidth=1, edgecolor='r')
    ax.add_patch(rect)
plt.show()


def resize_with_coordinates(image, width, height, coordinates):
    original_height, original_width = image.shape[:2]
    resized_image = cv2.resize(image, (width, height))
    
    new_coord = coordinates.copy()
    if coordinates is not None:
        rate_x = width / original_width
        rate_y = height / original_height
        for i,coord in enumerate(coordinates):
            new_coord[i] = coord * (rate_x, rate_y)
    return resized_image, new_coord

def get_origin_coordinates(resized, origin, coordinates):
    original_height, original_width = origin.shape[:2]
    re_height, re_width = resized.shape[:2]
    
    new_coord = coordinates.copy()
    if coordinates is not None:
        
        rate_x = original_width / re_width
        rate_y = original_height / re_height
        for i,coord in enumerate(coordinates):
            print(coord)
            new_coord[i] = coord * (rate_x, rate_y)
    return new_coord

In [ ]:
gt = np.concatenate([i['box'][np.newaxis,:,:]for i in boxes])

In [ ]:
gt.shape

In [ ]:
gt

In [ ]:
a,b=resize_with_coordinates(img,768,1024,gt)

In [ ]:
bbox = get_origin_coordinates(a,img,b)
fig, ax = plt.subplots(figsize=(30,30))
ax.imshow(img)

for word in bbox:
    diff_x,diff_y,point = calc_point(word)
    rect = patches.Rectangle(point,diff_x,diff_y,facecolor='none', linewidth=1, edgecolor='r')
    ax.add_patch(rect)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(30,30))
ax.imshow(a)

for word in b:
    diff_x,diff_y,point = calc_point(word)
    rect = patches.Rectangle(point,diff_x,diff_y,facecolor='none', linewidth=1, edgecolor='r')
    ax.add_patch(rect)
plt.show()

In [ ]:
from scipy import interpolate

def intersection(x, p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    if x2 == x1:
        return 0
    k = (x - x1) / (x2 - x1)
    return k * (y2 - y1) + y1


def midpoint(p1, p2, typed=float):
    return [typed((p1[0] + p2[0]) / 2), typed((p1[1] + p2[1]) / 2)]

def box2seg(image, boxes):
    height, width = image.shape[:2]
    mask = np.zeros((height, width), dtype=np.float32)
    seg = np.zeros((height, width), dtype=np.float32)
    points = []
    for box_index in range(boxes.shape[0]):
        box = boxes[box_index, :, :] # 4x2
        left_top = box[0]
        right_top = box[1]
        right_bottom = box[2]
        left_bottom = box[3]

        left = [(left_top[0] + left_bottom[0]) / 2, (left_top[1] + left_bottom[1]) / 2]
        right = [(right_top[0] + right_bottom[0]) / 2, (right_top[1] + right_bottom[1]) / 2]

        center = midpoint(left, right)
        points.append(midpoint(left, center))
        points.append(midpoint(right, center))

        poly = np.array([midpoint(left_top, center),
            midpoint(right_top, center),
            midpoint(right_bottom, center),
            midpoint(left_bottom, center)
            ])
        seg = cv2.fillPoly(seg, [poly.reshape(4, 1, 2).astype(np.int32)], (255,255,255))

    left_y = intersection(0, points[0], points[1])
    right_y = intersection(width, points[-1], points[-2])
    points.insert(0, [0, left_y])
    points.append([width, right_y])
    points = np.array(points)

    f = interpolate.interp1d(points[:, 0], points[:, 1], fill_value='extrapolate')
    xnew = np.arange(0, width, 1)
    ynew = f(xnew).clip(0, height-1)
    for x in range(width - 1):
        mask[int(ynew[x]), x] = 1
    return ynew.reshape(1, -1).round(), seg, mask

In [ ]:
c,d, mask = box2seg(a,b)

In [ ]:
mask.shape

In [ ]:
plt.imshow(mask)

In [ ]:
tc.shape

In [ ]:
data

In [ ]:


tensor_img = torch.from_numpy(a[np.newaxis,:,:,:]).permute(0,3,1,2).contiguous().to(torch.float)

res_back = back(tensor_img)
res_neck = neck(res_back)
tc = torch.from_numpy(c).to(torch.float)
td = torch.from_numpy(d).to(torch.float)

loss(res_neck,td.unsqueeze(0).unsqueeze(0),td)

In [ ]:
for i in range(4):
    print(res_back[i].shape)

In [ ]:
for i in res_neck.keys():
    print(res_neck[i].shape)

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(test['image'])

In [ ]:
plt.imshow(test['polygons'])

In [ ]:
plt.imshow(test['ignore_tags'])

In [ ]:
import matplotlib.patches as patches


fig, ax = plt.subplots(figsize=(30,30))
ax.imshow(test['image'])

for word in test['polygons']:
    diff_x,diff_y,point = calc_point(word)
    rect = patches.Rectangle(point,diff_x,diff_y,facecolor='none', linewidth=1, edgecolor='r')
    ax.add_patch(rect)
plt.show()

In [ ]:
plt.imshow(test['image'])

In [ ]:
data